In [ ]:
!pip install yfinance newsapi-python openai==0.28

In [ ]:
import yfinance as yf


crude = yf.Ticker("CL=F")

crude.history(period="7d", interval="1m")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-11-01 00:00:00-04:00,81.120003,81.120003,81.099998,81.110001,0,0.0,0.0
2023-11-01 00:01:00-04:00,81.110001,81.110001,81.099998,81.110001,40,0.0,0.0
2023-11-01 00:02:00-04:00,81.099998,81.099998,81.099998,81.099998,10,0.0,0.0
2023-11-01 00:03:00-04:00,81.089996,81.099998,81.089996,81.099998,5,0.0,0.0
2023-11-01 00:04:00-04:00,81.110001,81.129997,81.110001,81.110001,7,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-08 21:10:00-05:00,75.709999,75.739998,75.690002,75.709999,82,0.0,0.0
2023-11-08 21:11:00-05:00,75.699997,75.760002,75.690002,75.720001,63,0.0,0.0
2023-11-08 21:12:00-05:00,75.709999,75.720001,75.699997,75.699997,28,0.0,0.0


In [ ]:
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='')


all_articles = newsapi.get_everything(q='war',
                                      sources='bbc-news,the-verge, new-york-times',
                                      domains='bbc.co.uk,techcrunch.com,nytimes.com',
                                      from_param='2023-11-01',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)


In [ ]:
[article['content'] for article in all_articles['articles']]

['Donald Trump has devoted much of this presidential campaign looking back, contesting his 2020 election defeat. But behind the scenes, he and his team are putting together a plan for power, determined… [+9732 chars]',
 "McDonald's has condemned what it called disinformation over the Israel-Gaza war after boxes of live mice were released in three of its restaurants in apparent protests.\r\nA video on social media showe… [+2364 chars]",
 'A pro-Palestinian march due to take place on Armistice Day would only be banned as a "last resort", the Metropolitan Police Commissioner said.\r\nSir Mark Rowley vowed officers would do all they could … [+3386 chars]',
 "A ban on conversion therapy practices has not been included in the King's Speech, which sets out laws ministers want to pass in the year ahead.\r\nThe legislation, first promised in 2018, now looks unl… [+3188 chars]",
 'Andy McDonald has threatened a fellow MP with legal action, over a social media post about his speech at a pro-Pales

In [ ]:
all_articles['articles'][0]

{'source': {'id': 'bbc-news', 'name': 'BBC News'},
 'author': 'https://www.facebook.com/bbcnews',
 'title': 'Donald Trump second term: What would he do if he wins?',
 'description': 'If he returns to the White House by winning the 2024 election, he knows what he wants to do with the power.',
 'url': 'https://www.bbc.co.uk/news/world-us-canada-67272569',
 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/0F46/production/_131601930_trump_super_promo_2x_976x549-nc.png',
 'publishedAt': '2023-11-03T22:09:06Z',
 'content': 'Donald Trump has devoted much of this presidential campaign looking back, contesting his 2020 election defeat. But behind the scenes, he and his team are putting together a plan for power, determined… [+9732 chars]'}

In [ ]:
import openai

def answers_format():
    return "{'impact' : 'fill this'}"

def generate_recommendation(news, api_key):
    prompt = f"""given this news article\n{news}\n
    based on this news, can you give your opinion how important the news and its impact to the oil prices. give scale 1-10
    Give your answers similar to this format
    {answers_format()}
    """
    openai.api_key = api_key[0]
    openai.api_base = api_key[1]
    openai.api_type = 'azure'
    openai.api_version = '2023-08-01-preview'

    response = openai.Completion.create(
        engine="gpt-pwc",
        prompt=prompt,
        max_tokens=400,
        n=1,
        stop=None,
        temperature=0.15
    )

    recommendation_str = response.choices[0].text.strip()


    return recommendation_str

In [ ]:
import ast

openai_key = ''
openai_api_base = 'https://pwc-azure.openai.azure.com/'
print(f"Original news content: {all_articles['articles'][0]['content']}")
impact = generate_recommendation(all_articles['articles'][0]['content'], [openai_key, openai_api_base])
print(f"Impact: {ast.literal_eval(impact)['impact']}")

Original news content: Donald Trump has devoted much of this presidential campaign looking back, contesting his 2020 election defeat. But behind the scenes, he and his team are putting together a plan for power, determined… [+9732 chars]
Impact: 7
